# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

In [1]:
import numpy as np
import pandas as pd

## IZ

Загрузим датасет iz. Одновременно сделаем преобразование даты из строки

In [2]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [3]:
data_iz = pd.read_csv('/Users/marinadorosh/Desktop/openDS/iz.csv', 
                      escapechar='\\', 
                      parse_dates=['date'],
                      quotechar='\"', 
                      error_bad_lines=False,
                      index_col=False,)

b'Skipping line 61517: expected 17 fields, saw 20\n'


In [4]:
# Проверим преобразование
data_iz.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors                object
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [5]:
data_iz.shape

(86600, 16)

In [6]:
# Так выглядет датасет IZ
data_iz.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2017-01-24 18:24:00,https://iz.ru/699785/nataliia-portiakova/chtob...,-,Мир,Наталия Портякова,Чтобы помнили,В Европейском парламенте (ЕП) 24 января прошла...,-,-,-,-,-,-,-,-,-
1,2017-11-23 23:30:00,https://iz.ru/695670/izru-poluchil-premiiu-runeta,-,Пресс-релизы,NaN,IZ.ru получил премию Рунета,"IZ.ru стал победителем в номинации «Культура, ...",-,-,-,-,-,-,-,-,-
2,2017-11-25 23:49:47,https://iz.ru/675449/2017-11-25/chislo-mashin-...,-,Общество,NaN,Число машин с «мигалками» увеличили в Совфеде,"Президент России Владимир Путин подписал указ,...",-,-,-,-,-,-,-,-,-
3,2017-11-25 23:47:03,https://iz.ru/675448/2017-11-25/kudrin-zaiavil...,-,Экономика,NaN,Кудрин заявил о подготовке проекта развития РФ...,"Бывший министр финансов России, глава Центра с...",-,-,-,-,-,-,-,-,-
4,2017-11-25 23:46:48,https://iz.ru/675447/2017-11-25/gollandetc-kup...,-,Мир,NaN,Голландец купил акварель Гитлера за 75 центов,"Женщина, пожелавшая остаться неизвестной, отда...",-,-,-,-,-,-,-,-,-


In [ ]:
# Выясним в каким столбцах есть пустые данные
data_rt.isnull().any()

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [7]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_iz['text'].isnull().sum())
data_iz[data_iz['text'].isnull()]

Кол-во строк с значением NaN в столбце "Text" =  1268


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
330,2017-01-01 00:00:00,https://iz.ru/654049/video/odnopolye-braki,-,Общество,NaN,однополые браки,NaN,-,-,-,-,-,-,-,-,-
395,2017-09-12 22:43:46,https://iz.ru/644950/video/musor,-,Общество,NaN,Утилизация мусора,NaN,-,-,-,-,-,-,-,-,-
504,2017-09-12 18:28:25,https://iz.ru/644832/video/den-goroda-na-poklo...,-,Общество,NaN,День города на Поклонной горе. Александр Кибов...,NaN,-,-,-,-,-,-,-,-,-
506,2017-09-12 18:27:24,https://iz.ru/644830/video/den-goroda-na-poklo...,-,Общество,NaN,День города на Поклонной горе,NaN,-,-,-,-,-,-,-,-,-
581,2017-11-27 14:17:04,https://iz.ru/675938/video/proshchanie-s-dmitr...,-,Общество,NaN,Прощание с Дмитрием Хворостовским,NaN,-,-,-,-,-,-,-,-,-
624,2017-09-12 15:28:16,https://iz.ru/644731/video/grafika-sanktcii-oo...,-,Экономика,NaN,Графика. Санкции ООН против КНДР,NaN,-,-,-,-,-,-,-,-,-
639,2017-11-27 14:35:45,https://iz.ru/675930/video/android-drug-cheloveka,-,Общество,NaN,АНДРОИД — ДРУГ ЧЕЛОВЕКА,NaN,-,-,-,-,-,-,-,-,-
677,2017-11-27 13:13:52,https://iz.ru/675889/video/chempionat-mira-gra...,-,Спорт,NaN,Чемпионат мира графика,NaN,-,-,-,-,-,-,-,-,-
1064,2017-09-13 14:01:20,https://iz.ru/645175/video/video-zaderzhaniia-...,-,Общество,NaN,Задержание замдиректора ФСИН России Коршунова,NaN,-,-,-,-,-,-,-,-,-
1106,2017-11-27 04:31:20,https://iz.ru/675737/video/grafika-sochi,-,Спорт,NaN,графика Сочи,NaN,-,-,-,-,-,-,-,-,-


In [8]:
#Удалим эти строки
data_iz['text'].dropna(inplace=True)

In [9]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_iz['text'].isnull().sum())

Кол-во строк с значением NaN в столбце "Text" =  0


Посмотрим сколько элементов Nan в столбце авторов

In [15]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_iz['authors'].isnull().sum())
print('Процент статтей без автора: ', np.round(data_iz['authors'].isnull().sum()/data_iz.shape[0] *100, 2))

Кол-во строк с значением NaN в столбце "authors" =  62959
Процент статтей без автора:  72.7


Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [17]:
print('edition: ',data_iz['edition'].unique())
print('reposts_fb: ', data_iz['reposts_fb'].unique())
print('reposts_vk: ',data_iz['reposts_vk'].unique())
print('reposts_ok: ',data_iz['reposts_ok'].unique())
print('reposts_twi: ',data_iz['reposts_twi'].unique())
print('reposts_lj: ',data_iz['reposts_lj'].unique())
print('reposts_tg: ',data_iz['reposts_tg'].unique())
print('likes: ',data_iz['likes'].unique())
print('views: ',data_iz['views'].unique())
print('comm_count: ',data_iz['comm_count'].unique())

edition:  ['-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


Значений нет, можно удалять

### Рубрики IZ

In [20]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_iz.topics.unique().shape)
data_iz['topics'].unique()

Количество рубрик =  (34,)


array(['Мир', 'Пресс-релизы', 'Общество', 'Экономика', 'Спорт', 'Наука',
       'Политика', 'Культура', 'Армия', 'Страна', 'Авто', 'Интернет',
       'Образование', 'Потребительский рынок', 'Новости компаний',
       'День в истории', 'Туризм', 'Вкусно!', 'Россия - регионы',
       'Недвижимость', 'Китай', 'Art Briz', 'Здоровье', 'Среда для жизни',
       'Транспорт', 'Стиль', 'Происшествия', 'Финансы', 'Инновации',
       'Финансовая грамотность', 'Профсоюзы', 'Спецпроекты',
       'Деловая среда', 'День Победы'], dtype=object)

Все рубрики, с количеством новостей в каждой группе

In [23]:
data_iz.groupby('topics')['topics'].count().sort_values()

topics
Деловая среда                 1
День Победы                   2
Спецпроекты                   2
Профсоюзы                     3
Пресс-релизы                  3
Стиль                         4
Финансовая грамотность        6
Происшествия                  6
Вкусно!                       8
Art Briz                      9
Потребительский рынок         9
Китай                        14
Среда для жизни              15
Финансы                      21
Новости компаний             21
Образование                  26
Недвижимость                 26
День в истории               28
Инновации                    42
Авто                         48
Россия - регионы             48
Здоровье                     50
Транспорт                    63
Туризм                       79
Интернет                    911
Наука                      1150
Армия                      1221
Культура                   5002
Спорт                      5788
Страна                     7736
Экономика                  7828
П